In [1]:
#!pip3 install utils
import ujson as json
import os
%matplotlib inline
import utils

In [2]:
from utils import *
from __future__ import division, print_function
from shutil import copyfile
from keras.utils import get_file
from sklearn.metrics import log_loss
import pandas as pd

Using TensorFlow backend.


In [3]:
import cv2
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical


/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def get_image_vector(image,size=(64,64)):
    return cv2.resize(image,size).flatten()

In [5]:
# Extracts the color histogram from images
def extract_color_histogram(image,bins = (8,8,8)):
    hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv],[0,1,2],None,bins,[0,256,0,256,0,256])
    cv2.normalize(hist,hist)
    return hist.flatten()

In [6]:
train_dir = "../train/"
test_dir = "../test_stg1"

In [7]:
classes = os.listdir(train_dir)
classes = [x for x in os.listdir(train_dir) if not x.startswith('.')]

In [8]:
image_path_list = []
for c in classes:
    fish_images = [train_dir+c+'/'+item for item in os.listdir(train_dir+c+'/')]
    image_path_list.extend(fish_images)

In [9]:

labels = []
for c in classes:
    l = [c]*len(os.listdir(train_dir+c+'/'))
    labels.extend(l)

In [10]:
labels = LabelEncoder().fit_transform(labels)

In [12]:
features = []
for i,image_path in enumerate(image_path_list):
    image = cv2.imread(image_path)
    hist = extract_color_histogram(image)
    features.append(hist)
    if(i%1000==0):
        print(str(i)+ "  completed")


0  completed
1000  completed
2000  completed
3000  completed


In [13]:
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size = 0.30, random_state = 42)

In [14]:
print (type(X_train))

<class 'list'>


In [ ]:
model = KNeighborsClassifier(n_neighbors = 4, n_jobs = -1)

In [ ]:
model.fit(X_train,y_train)

In [17]:
accuracy = model.score(X_test,y_test)

In [18]:
print ('accuracy is',(accuracy))

0.853615520282


In [19]:
preds_validation = model.predict_proba(X_test)

In [20]:
log_loss(y_test,preds_validation)

1.5871487636464505

In [21]:
test_files = [im for im in os.listdir(test_dir)]

In [22]:
test_dir +'/'+ test_files[0]

'../test_stg1/img_01237.jpg'

In [23]:
test_features = []
for i,image_path in enumerate(test_files):
    image = cv2.imread(test_dir + '/'+ image_path)
    hist = extract_color_histogram(image)
    test_features.append(hist)
    if(i%1000==0):
        print(str(i)+ "  completed")

0  completed


In [24]:
preds = model.predict_proba(test_features)